In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

### Part I Scrape data from wikipedia

In [2]:
# url of wikipedia page: List of postal codes of Canada: M
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# use beautifulsoup get the page content
soup = bs(requests.get(url).text,'lxml')

In [4]:
# find out data table
table = soup.find(class_="wikitable sortable")

In [5]:
# get table header
header = [x.get_text().strip() for x in table.findAll('tr')[0].findAll('th')]

In [6]:
# get table content
content = [[y.get_text().strip() for y in x.findAll('td')] for x in table.findAll('tr')[1:]]

In [7]:
# build padas dataframe
df = pd.DataFrame(data=content, columns=header)

In [8]:
# remove 'Not assigned' Borough
df = df[df['Borough']!='Not assigned']

In [9]:
# reindex dataframe
df = df.reset_index(drop=True)

In [10]:
# if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for i in df.index:
    df['Neighbourhood'][i] = df['Borough'][i] if df['Neighbourhood'][i] == 'Not assigned' else df['Neighbourhood'][i]

In [11]:
# stack neighborhood use groupby and separate by comma
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x:x+',')
df = df.groupby(['Postcode','Borough'], as_index=False)['Neighbourhood'].sum()
df['Neighbourhood'] = df['Neighbourhood'].apply(lambda x:x[:-1])

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df.shape

(103, 3)

### Part II Add coordinates

In [14]:
# read coordinates data
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [15]:
# add coordinates data to dataframe
df = df.merge(geo[['Postal Code','Latitude','Longitude']], left_on=['Postcode'], right_on=['Postal Code'], how='left')
df = df.drop(['Postal Code'],axis=1)

In [16]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part III Segment and cluster

In [86]:
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [20]:
# get the coordinates of Toronto
geolocator = Nominatim(user_agent='Toronto')
address ='Toronto, Canada'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [21]:
latitude,longitude

(43.653963, -79.387207)

In [26]:
# create map of Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [27]:
# define foursquare credentials and version
CLIENT_ID = '*******'
CLIENT_SECRET = '*******'
VERSION = '20180605'

In [30]:
# the function of neighborhood venues
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# use function to create dataframe of Toronto_venues
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                                )

In [33]:
# check if the dataframe is correct
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [34]:
# size of dataframe
toronto_venues.shape

(2247, 7)

In [50]:
# number of venues returned for each neighborhood
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",10,10,10,10,10,10


In [37]:
# unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


#### Analyze Each Neighborhood

In [51]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], 
                                  prefix="", 
                                  prefix_sep=""
                                 )

In [52]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

In [53]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [54]:
# check if the dataframe is correct
toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# size of dataframe
toronto_onehot.shape

(2247, 277)

In [60]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [61]:
# check if the dataframe is correct
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [62]:
# size of dataframe
toronto_grouped.shape

(100, 277)

In [63]:
# function to get the top N venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
# get top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [67]:
# check if the dataframe is correct
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,American Restaurant,Steakhouse,Café,Thai Restaurant,Asian Restaurant,Restaurant,Burger Joint,Bar,Bakery
1,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Sculpture Garden,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Liquor Store,General Travel,General Entertainment
4,"Alderwood,Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Coffee Shop,Gay Bar


#### Cluster Neighborhoods

In [76]:
# Run k-means to cluster the neighborhood into 5 clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [77]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 4, 1, 2, 2, 2, 2, 2, 2])

In [83]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.dropna(inplace=True)

In [92]:
# check if the dataframe is correct
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Moving Target,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2.0,Electronics Store,Medical Center,Mexican Restaurant,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Doner Restaurant


In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,0.0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store
97,North York,0.0,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Department Store


#### Cluster 2

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
15,Scarborough,1.0,Fast Food Restaurant,Chinese Restaurant,Coffee Shop,Breakfast Spot,Indian Restaurant,Electronics Store,Discount Store,Cosmetics Shop,Pharmacy,Pizza Place
25,North York,1.0,Park,Fast Food Restaurant,Pool,Food & Drink Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
31,North York,1.0,Grocery Store,Moving Target,Shopping Mall,Bank,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
33,North York,1.0,Gym / Fitness Center,Grocery Store,Liquor Store,Discount Store,Athletics & Sports,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
81,York,1.0,Grocery Store,Pizza Place,Brewery,Caribbean Restaurant,Convenience Store,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
92,Etobicoke,1.0,Grocery Store,Convenience Store,Fast Food Restaurant,Burger Joint,Sandwich Place,Discount Store,Bakery,Supplement Shop,Tanning Salon,Wings Joint
101,Etobicoke,1.0,Grocery Store,Pizza Place,Fried Chicken Joint,Pharmacy,Sandwich Place,Beer Store,Fast Food Restaurant,Liquor Store,General Travel,General Entertainment


#### Cluster 3

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2.0,Bar,Moving Target,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
2,Scarborough,2.0,Electronics Store,Medical Center,Mexican Restaurant,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Dim Sum Restaurant,Diner
3,Scarborough,2.0,Coffee Shop,Korean Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,Scarborough,2.0,Hakka Restaurant,Fried Chicken Joint,Lounge,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Athletics & Sports,Donut Shop,Doner Restaurant
5,Scarborough,2.0,Convenience Store,Playground,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
6,Scarborough,2.0,Discount Store,Convenience Store,Bus Station,Coffee Shop,Department Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,Scarborough,2.0,Bus Line,Bakery,Soccer Field,Intersection,Fast Food Restaurant,Bus Station,Metro Station,Dog Run,Diner,Discount Store
8,Scarborough,2.0,Motel,Skating Rink,American Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
9,Scarborough,2.0,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,Scarborough,2.0,Indian Restaurant,Chinese Restaurant,Light Rail Station,Latin American Restaurant,Furniture / Home Store,Pet Store,Vietnamese Restaurant,Comfort Food Restaurant,Department Store,Electronics Store


#### Cluster 4

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,North York,3.0,Pizza Place,Empanada Restaurant,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
100,Etobicoke,3.0,Pizza Place,Mobile Phone Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant


#### Cluster 5

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4.0,Park,Playground,Sculpture Garden,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
20,North York,4.0,Cafeteria,Park,Martial Arts Dojo,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
23,North York,4.0,Park,Bank,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
30,North York,4.0,Park,Bus Stop,Airport,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
40,East York,4.0,Park,Convenience Store,Coffee Shop,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
44,Central Toronto,4.0,Park,Bus Line,Swim School,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Deli / Bodega
50,Downtown Toronto,4.0,Park,Trail,Playground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
64,Central Toronto,4.0,Park,Trail,Sushi Restaurant,Jewelry Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,York,4.0,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant
79,North York,4.0,Basketball Court,Park,Bakery,Construction & Landscaping,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
